# Why Dummy data?

In the real dataset a lot of the reservoirs lack inflow data. Hence, resorting to dummy data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

In [2]:
# # reservoir A
# A = {}
# A['inflow'] = np.array([100, 120, 130, 90, 60, 30, 20, 30, 20, 90, 150, 100, 80, 70, 50, 40])
# A['storage_change'] = np.array([np.nan, 40, 40, 10, 0, -10, -10, -20, -20])

# Dummy data

In [1]:
from landlab import NetworkModelGrid

In [2]:
y_of_node = (0, 1, 2, 2)
x_of_node = (0, 0, -1, 1)

nodes_at_link=(
    (1, 0),
    (2, 1),
    (3, 1)
)
grid = NetworkModelGrid((y_of_node, x_of_node), nodes_at_link)
grid

<xarray.Dataset>
Dimensions:        (node: 4, link: 3, Two: 2)
Coordinates:
  * node           (node) int64 0 1 2 3
Dimensions without coordinates: link, Two
Data variables:
    mesh           <U1 'a'
    y_of_node      (node) float64 0.0 1.0 2.0 2.0
    x_of_node      (node) float64 0.0 0.0 -1.0 1.0
    nodes_at_link  (link, Two) int64 0 1 2 1 1 3

In [33]:
inflow = grid.add_field(
    'reservoir__unregulated_inflow',
    [3, 2, 1, 1],
    at = 'node',
    units='m3/d',
    clobber=True
)
storage_change = grid.add_field(
    'reservoir__storage_change',
    [0.3, 0.2, -0.1, 0.1],
    at = 'node',
    units='m3/d',
    clobber=True
)
release = grid.add_field(
    'reservoir__release',
    [2.7, 1.8, 1.1, 0.9],
    at = 'node',
    units = 'm3/d',
    clobber=True
)
abstract_elevation = grid.add_field(
    'topographic__elevation',
    [0.0, 1.0, 2.0, 2.0],
    at = 'node',
    units = 'm',
    clobber=True
)
cell_area_at_node = grid.add_field(
    'cell_area_at_node',
    [1, 1, 1, 1],
    at = 'node',
    units = 'm2',
    clobber=True
)


In [34]:
from landlab.components import FlowDirectorSteepest

fdr = FlowDirectorSteepest(grid)
fdr.run_one_step()
fdr

In [23]:
for node in grid.nodes:
    print(grid.adjacent_nodes_at_node[node])

[ 1 -1 -1]
[3 2 0]
[ 1 -1 -1]
[ 1 -1 -1]


In [20]:
grid.links_at_node[0]

array([ 0, -1, -1])

In [105]:
import numpy as np
import numpy.ma as ma

In [116]:
def get_upstream_inflow(grid):
    """Calculates the upstream inflow

    Args:
        grid (_type_): _description_
        node (_type_): _description_

    Returns:
        _type_: _description_
    """
    fdr = FlowDirectorSteepest(grid, abstract_elevation)
    fdr.run_one_step()

    upstream_contributing_links_at_node = np.where(
        fdr.flow_link_incoming_at_node() == 1, grid.links_at_node, -1
    )

    upstream_links = ma.MaskedArray(upstream_contributing_links_at_node, mask=upstream_contributing_links_at_node==-1)
    upstream_nodes = ma.MaskedArray(fdr.upstream_node_at_link()[upstream_links], mask=upstream_links.mask)
    
    regulated_inflow = ma.sum(
        ma.MaskedArray(grid.at_node['reservoir__release'][upstream_nodes], mask=upstream_links.mask), axis=1
    ).filled(0)

    return regulated_inflow

get_upstream_inflow(grid)

array([ 1.8,  2. ,  0. ,  0. ])

# Using StreamflowRegulation component

In [77]:
%load_ext autoreload
%autoreload 2
import numpy as np

from landlab import NetworkModelGrid
y_of_node = (0, 1)#, 2, 2)
x_of_node = (0, 0)#, -1, 1)

nodes_at_link=(
    (1, 0),
    # (2, 1),
    # (3, 1)
)
grid = NetworkModelGrid((y_of_node, x_of_node), nodes_at_link)


grid.add_field(
    "reservoir__total_inflow",
    np.full_like(grid.x_of_node, np.nan),
    at='node',
    units='m3/d' ,
    clobber=True
)
grid.add_field(
    "reservoir__storage_change",
    np.full_like(grid.x_of_node, np.nan),
    at='node',
    units='m3/d' ,
    clobber=True
)
grid.add_field(
    "reservoir__release",
    np.full_like(grid.x_of_node, np.nan),
    at='node',
    units='m3/d' ,
    clobber=True
)
grid.add_field(
    "reservoir__regulated_inflow",
    np.full_like(grid.x_of_node, np.nan),
    at='node',
    units='m3/d' ,
    clobber=True
)
grid.add_field(
    "reservoir__unregulated_inflow",
    np.full_like(grid.x_of_node, np.nan),
    at='node',
    units='m3/d' ,
    clobber=True
)
grid.add_field(
    "reservoir__abstract_elevation",
    (0.0, 1.0),
    at='node',
    units='m3/d' ,
    clobber=True
)
import numpy as np

from landlab.data_record import DataRecord
from reservoirnetwork.reservoir_component import StreamflowRegulation
items = None

time = [ 1, 2, 3, 4, 5]#, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

r_A_inflow = [ 120, 130, 90, 60, 30]#, 20, 30, 20, 15, 15, 10, 9, 9, 9, 8, 9, 90, 150, 120, 100, 80, 70, 50, 40, 30, 23, 16, 10]
r_A_storage_change = [ 30, 30, 10, 0, 0]#, -5, 0, -10, -15, -15, -15, -10, -10, -8, -4, -5, 5, 40, 35, 20, 10, 3, -4, -10, -20, -10, -15, -20]
r_B_inflow = [ 162, 150, 108, 63, 40.5]#, 39, 45, 39, 36, 31.5, 23.1, 22.5, 20.7, 16.2, 17.4, 81.9, 153, 166.5, 144, 123, 108.3, 90.6, 75, 69, 47.7, 41.7, 36.6, 6]
r_B_storage_change = [ 35, 30, 30, 10, 0]#, 0, 10, -10, -15, -20, -20, -20, -15, -15, -20, 10, 40, 40, 25, 20, 5, -10, -20, -15, -16, -15, -10, -20]
last_release = [90, 127]

grid.at_node['reservoir__release'] = last_release

# fluxes = DataRecord(
#     grid,
#     time=time,
#     items={
#         ["node"],
#         [0, 1]
#     }
#     data_vars={
#         'total_inflow' : (time, [r_A_inflow, r_B_inflow])
#     }
# )

sr = StreamflowRegulation(grid)

data = {output_var_name: [] for output_var_name in sr.output_var_names}

for t, inflow, storage_change in zip(
    time, 
    zip(r_A_inflow, r_B_inflow), 
    zip(r_A_storage_change, r_B_storage_change)
):
    # grid.at_node['reservoir__release'] = last_release
    sr.run_one_step(t, inflow, storage_change)

    last_release = grid.at_node['reservoir__release']
    print(grid.at_node['reservoir__release'])
    # print(grid.at_node['reservoir__unregulated_inflow'])
    # print(grid.at_node['reservoir__regulated_inflow'], grid.at_node['reservoir__total_inflow'])
    # print(last_release)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[ 90 127]
[100 120]
[80 78]
[60 53]
[ 30.   40.5]
